In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

In [2]:
# SETUP

G_INPUT_SIZE = 1
D_INPUT_SIZE = 1
D_OUTPUT_SIZE = 1
HIDDEN_SIZE = 11

BATCH_SIZE = 100
LEARNING_RATE = 0.01
EPOCHS = 10 ** 6

In [3]:
# INIT

def z_sample(shape):
    return np.random.uniform(0, 1, shape)


def x_sample(shape):
    return np.random.normal(-1, 1, shape)


def generator(z):
    hidden = fully_connected(z, HIDDEN_SIZE, activation_fn=tf.sigmoid)
    output = fully_connected(hidden, D_INPUT_SIZE, activation_fn=tf.sigmoid)
    return output


def discriminator(x):
    hidden = fully_connected(x, HIDDEN_SIZE, activation_fn=tf.sigmoid)
    output = fully_connected(hidden, D_OUTPUT_SIZE, activation_fn=tf.sigmoid)
    return output

In [4]:
with tf.device('/gpu:0'):
    with tf.variable_scope('G'):
        z = tf.placeholder(tf.float32, shape=(None, G_INPUT_SIZE))
        G = generator(z)

    with tf.variable_scope('D') as scope:
        x = tf.placeholder(tf.float32, shape=(None, D_INPUT_SIZE))
        D1 = discriminator(x)
        D2 = discriminator(G)

        loss_d = tf.reduce_mean(-tf.log(D1) - tf.log(1 - D2))
        loss_g = tf.reduce_mean(-tf.log(D2))

        trainable_vars = tf.trainable_variables()
        d_params = [v for v in trainable_vars if v.name.startswith('D/')]
        g_params = [v for v in trainable_vars if v.name.startswith('G/')]
        print("D params count: {}".format(len(d_params)))
        print("G params count: {}".format(len(g_params)))

        opt_d = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss_d)
        opt_g = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss_g)
        
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as session:
    session.run(tf.initialize_all_variables())

    for step in range(EPOCHS):
        zs = z_sample([BATCH_SIZE, G_INPUT_SIZE])
        loss_gs, _ = session.run([loss_g, opt_g], feed_dict={z: zs})

        xs = x_sample([BATCH_SIZE, D_INPUT_SIZE])
        zs = z_sample([BATCH_SIZE, G_INPUT_SIZE])
        loss_ds, _ = session.run([loss_d, opt_d], feed_dict={x: xs, z: zs})

        if step % 10000 == 0:
            print('Epoch {}: loss_d={} loss_g={}'.format(step, loss_ds, loss_gs))

D params count: 8
G params count: 4


FailedPreconditionError: Attempting to use uninitialized value D/fully_connected_3/biases
	 [[Node: D/fully_connected_3/biases/read = Identity[T=DT_FLOAT, _class=["loc:@D/fully_connected_3/biases"], _device="/job:localhost/replica:0/task:0/cpu:0"](D/fully_connected_3/biases)]]

Caused by op 'D/fully_connected_3/biases/read', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super